In [12]:
import os
import numpy as np
import pandas as pd
import re
from datetime import datetime, timedelta

root_path = '../../../data/poteka-raw-data/'
obpoint_folders = os.listdir(root_path)

# year = '2020'
# monthes = ['07', '08', '09', '10']
# good_files = 0
# bad_files = 0
# for month in monthes:
#     path = root_path + obpoint_folders[0] + '/2020/' + month
#     for dates in os.listdir(path):
#         csv_path = path + '/' + dates + '/Weather/'
#         # check the weather csv file exists
#         if len(os.listdir(csv_path)) > 0:
#             filename = os.listdir(csv_path)[0]
#             df = pd.read_csv(csv_path + filename)
#             good_files += 1
#             if len(df) < 1440:
#                 bad_files += 1
#                 print(dates, 1440 - len(df))
                
# print(good_files, bad_files)


path = '../../../data/poteka-raw-data/Vlz-Bagbaguin_00174731/2020/07/2020-07-21/Weather/weather_00174731_20200721.csv'
df_before = pd.read_csv(path, header=None)

cols = ['Datetime', 'RAF', 'RA1', 'RI1', 'ERA', 'CRA', 'AT1', 'RH1', 'PRS', 'SLP', 'WD1', 'WDM', 'WS1', 
        'WSM', 'WND', 'WNS', 'SOL', 'WET', 'WBG', 'WEA']

def find_value(x):
    result = re.findall('\d+', x)
    if len(result) > 1:
        return result[-1]
    elif len(result) == 1:
        return result[0]
    else:
        return np.nan

for col in cols:
    for col_before in df_before.columns:
        # ra1, raf, ...
        if col in str(df_before[col_before].values[0]):
            df_before[col] = df_before[col_before].apply(find_value)
            df_before[col] = df_before[col].apply(float)
            
def format_datetime(x):
    x = str(x)
    year = int(x[:4])
    month = int(x[4:6])
    date = int(x[6:8])
    hour = int(x[8:10])
    minute = int(x[10:12])
    dt = datetime(year, month, date, hour, minute)
    return dt.strftime('%Y-%m-%d T%H:%M Z')
            
for col in df_before.columns:
    val = df_before[col].values[0]
    check = find_value(str(val))
    if len(check) == 14:
        df_before["Datetime"] = df_before[col].apply(str)
        df_before["Datetime"] = df_before["Datetime"].apply(format_datetime)
        

            
df_before = df_before[cols]
df_before = df_before.set_index('Datetime')
df_before.index

Index(['2020-07-21 T02:13 Z', '2020-07-21 T02:12 Z', '2020-07-21 T02:11 Z',
       '2020-07-21 T02:16 Z', '2020-07-21 T02:15 Z', '2020-07-21 T02:14 Z',
       '2020-07-21 T02:17 Z', '2020-07-21 T02:19 Z', '2020-07-21 T02:18 Z',
       '2020-07-21 T02:20 Z',
       ...
       '2020-07-21 T13:08 Z', '2020-07-21 T13:07 Z', '2020-07-21 T13:06 Z',
       '2020-07-21 T13:05 Z', '2020-07-21 T13:04 Z', '2020-07-21 T13:03 Z',
       '2020-07-21 T13:02 Z', '2020-07-21 T13:01 Z', '2020-07-21 T13:00 Z',
       '2020-07-21 T12:59 Z'],
      dtype='object', name='Datetime', length=856)

In [13]:
cols = ['RAF', 'RA1', 'RI1', 'ERA', 'CRA', 'AT1', 'RH1', 'PRS', 'SLP', 'WD1', 'WDM', 'WS1', 
        'WSM', 'WND', 'WNS', 'SOL', 'WET', 'WBG', 'WEA']
magni = [10, 10, 10, 10, 10, 10, 10, 10, 10, 1, 1, 10, 10, 1, 10, 1, 1, 10, 1]
for i in range(len(cols)):
    col = cols[i]
    mag = magni[i]
    df_before[col] = df_before[col] / mag
df_before.index

Index(['2020-07-21 T02:13 Z', '2020-07-21 T02:12 Z', '2020-07-21 T02:11 Z',
       '2020-07-21 T02:16 Z', '2020-07-21 T02:15 Z', '2020-07-21 T02:14 Z',
       '2020-07-21 T02:17 Z', '2020-07-21 T02:19 Z', '2020-07-21 T02:18 Z',
       '2020-07-21 T02:20 Z',
       ...
       '2020-07-21 T13:08 Z', '2020-07-21 T13:07 Z', '2020-07-21 T13:06 Z',
       '2020-07-21 T13:05 Z', '2020-07-21 T13:04 Z', '2020-07-21 T13:03 Z',
       '2020-07-21 T13:02 Z', '2020-07-21 T13:01 Z', '2020-07-21 T13:00 Z',
       '2020-07-21 T12:59 Z'],
      dtype='object', name='Datetime', length=856)

In [14]:
# check droping time
# timescale yyyymmdd000000 ~ yyyymmdd235900 skipping 100
def datetime_range(start, end, delta):
    current = start
    while current <= end:
        yield current
        current += delta
        

def create_time_range(year, month, date):
    dts = [dt.strftime('%Y-%m-%d T%H:%M Z') for dt in
           datetime_range(datetime(year, month, date, 0), datetime(year, month, date, 23, 59), timedelta(minutes=1))]
    return dts

times = create_time_range(2020, 7, 21)

df_after = df_before.reindex(times)
df_before.index

Index(['2020-07-21 T02:13 Z', '2020-07-21 T02:12 Z', '2020-07-21 T02:11 Z',
       '2020-07-21 T02:16 Z', '2020-07-21 T02:15 Z', '2020-07-21 T02:14 Z',
       '2020-07-21 T02:17 Z', '2020-07-21 T02:19 Z', '2020-07-21 T02:18 Z',
       '2020-07-21 T02:20 Z',
       ...
       '2020-07-21 T13:08 Z', '2020-07-21 T13:07 Z', '2020-07-21 T13:06 Z',
       '2020-07-21 T13:05 Z', '2020-07-21 T13:04 Z', '2020-07-21 T13:03 Z',
       '2020-07-21 T13:02 Z', '2020-07-21 T13:01 Z', '2020-07-21 T13:00 Z',
       '2020-07-21 T12:59 Z'],
      dtype='object', name='Datetime', length=856)

In [4]:
df_after.to_csv('test.csv')

In [15]:
df_before.index

Index(['2020-07-21 T02:13 Z', '2020-07-21 T02:12 Z', '2020-07-21 T02:11 Z',
       '2020-07-21 T02:16 Z', '2020-07-21 T02:15 Z', '2020-07-21 T02:14 Z',
       '2020-07-21 T02:17 Z', '2020-07-21 T02:19 Z', '2020-07-21 T02:18 Z',
       '2020-07-21 T02:20 Z',
       ...
       '2020-07-21 T13:08 Z', '2020-07-21 T13:07 Z', '2020-07-21 T13:06 Z',
       '2020-07-21 T13:05 Z', '2020-07-21 T13:04 Z', '2020-07-21 T13:03 Z',
       '2020-07-21 T13:02 Z', '2020-07-21 T13:01 Z', '2020-07-21 T13:00 Z',
       '2020-07-21 T12:59 Z'],
      dtype='object', name='Datetime', length=856)

In [5]:
import os
import numpy as np
import pandas as pd
import re
from datetime import datetime, timedelta

# Create the list of the every 1 minute datetime of a day
# Parameter
# ======================
# start: datetime of start of the day e.g. datetime(2020, 7, 21, 0)
# end: datetime of the end of the day e.g. datetime(2020, 7, 21, 23, 59)
# delta: integer how many times steps to next index e.g. 1 
def datetime_range(start, end, delta):
    current = start
    while current <= end:
        yield current
        current += delta

# Create Formated datetime list
# Parameter
# ========================
# year: the number of the year
# month: ...
# date: ...
def create_time_list(year, month, date):
    dts = [dt.strftime('%Y-%m-%d T%H:%M Z') for dt in 
            datetime_range(datetime(year, month, date, 0), datetime(year, month, date, 23, 59), timedelta(minutes=1))]
    return dts

# Find weather values from 'RAF 0000' -> 0.0
# Parameter
# =========================
# x: the value of cells of csv file
def find_value(x):
    result = re.findall('\d+', str(x))
    if len(result) > 1:
        return result[-1]
    elif len(result) == 1:
        return result[0]
    else:
        return np.nan

# Format 14 digir number to datetime
# Parameter
# ==========================
# x: the value of cells of csv file
def format_datetime(x):
    x = str(x)
    year = int(x[:4])
    month = int(x[4:6])
    date = int(x[6:8])
    hour = int(x[8:10])
    minute = int(x[10:12])
    dt = datetime(year, month, date, hour, minute)
    return dt.strftime('%Y-%m-%d T%H:%M Z')

# Find correct format csv file of the data
# Parameter
# ==========================
# path: {string} the path of the directory that the csv file is in it
# output: {string} the name of the csv file
def find_csv_file(path):
    files = os.listdir(path)
    result = []
    for file in files:
        for item in re.findall('^weather.+.csv$', str(file)):
            result.append(item)
    if len(result) > 1:
        return result[-1]
    else:
        return result[0]
    return result



cols = ['Datetime', 'RAF', 'RA1', 'RI1', 'ERA', 'CRA', 'AT1', 'RH1', 'PRS', 'SLP', 'WD1', 'WDM', 'WS1', 
        'WSM', 'WND', 'WNS', 'SOL', 'WET', 'WBG', 'WEA']


# magnification of weather values
magni = [10, 10, 10, 10, 10, 10, 10, 10, 10, 1, 1, 10, 10, 1, 10, 1, 1, 10, 1]

csv_path = '../../../data/poteka-raw-data/Sinegue_00181300/2020/05/2020-05-31/Weather/'

print('-'*50)
print(csv_path)

filename = find_csv_file(csv_path)
df_before = pd.read_csv(csv_path + filename, header=None)
print(df_before.info())

# Extract number of the data from the cell's value
for col in cols:
    for col_def in df_before.columns:
        if col in str(df_before[col_def].values[0]):
            df_before[col] = df_before[col_def].apply(find_value)
            df_before[col] = df_before[col].apply(float)
df_before.dropna(inplace=True)

# Extract datetime and format correctlly
if df_before.values.size > 0:
    for col in df_before.columns:
        val = df_before[col].values[0]
        check = find_value(str(val))
        if len(check) == 14:
            y = int(str(val)[:4])
            m = int(str(val)[4:6])
            d = int(str(val)[6:8])
            df_before['Datetime'] = df_before[col].apply(str)
            df_before['Datetime'] = df_before['Datetime'].apply(format_datetime)

    df_before = df_before[cols]
    df_before = df_before.set_index('Datetime')

    # Rescale data
    for i in range(len(cols[1:])):
        col = cols[i+1]
        mag = magni[i]
        df_before[col] = df_before[col] / mag

    # Reset index if there is a lack of the data
    time_list = create_time_list(y, m, d)
    df_before = df_before.drop_duplicates()
    print(len(time_list), len(df_before.index))
    print(df_before.info())
    df = df_before.reindex(time_list)

    save_path = '../../../data/cleaned-data/'
    oymd = [observation_point, year, month, date]
    for item in oymd:
        save_path += f'{str(item)}/'
        if not os.path.exists(save_path):
            os.mkdir(save_path)


    df.to_csv(save_path + 'data.csv')
else:
    print('Good Bye')

--------------------------------------------------
../../../data/poteka-raw-data/Sinegue_00181300/2020/05/2020-05-31/Weather/
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1093 entries, 0 to 1092
Data columns (total 21 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       1093 non-null   int64 
 1   1       1093 non-null   int64 
 2   2       1093 non-null   object
 3   3       1093 non-null   object
 4   4       1093 non-null   object
 5   5       1093 non-null   object
 6   6       1093 non-null   object
 7   7       1093 non-null   object
 8   8       1093 non-null   object
 9   9       1093 non-null   object
 10  10      1093 non-null   object
 11  11      1093 non-null   object
 12  12      1093 non-null   object
 13  13      1093 non-null   object
 14  14      1093 non-null   object
 15  15      1093 non-null   object
 16  16      1093 non-null   object
 17  17      1093 non-null   object
 18  18      1093 non-null   object
 19  19    

In [80]:
import os
import numpy as np
import pandas as pd
import re
from datetime import datetime, timedelta

# Create the list of the every 1 minute datetime of a day
# Parameter
# ======================
# start: datetime of start of the day e.g. datetime(2020, 7, 21, 0)
# end: datetime of the end of the day e.g. datetime(2020, 7, 21, 23, 59)
# delta: integer how many times steps to next index e.g. 1 
def datetime_range(start, end, delta):
    current = start
    while current <= end:
        yield current
        current += delta

# Create Formated datetime list
# Parameter
# ========================
# year: the number of the year
# month: ...
# date: ...
def create_time_list(year, month, date):
    dts = [dt.strftime('%Y-%m-%d T%H:%M Z') for dt in 
            datetime_range(datetime(year, month, date, 0), datetime(year, month, date, 23, 59), timedelta(minutes=1))]
    return dts

# Find weather values from 'RAF 0000' -> 0.0
# Parameter
# =========================
# x: the value of cells of csv file
def find_value(x):
    result = re.findall('\d+', str(x))
    if len(result) > 1:
        return result[-1]
    elif len(result) == 1:
        return result[0]
    else:
        return np.nan

# Format 14 digir number to datetime
# Parameter
# ==========================
# x: the value of cells of csv file
def format_datetime(x):
    x = str(x)
    year = int(x[:4])
    month = int(x[4:6])
    date = int(x[6:8])
    hour = int(x[8:10])
    minute = int(x[10:12])
    dt = datetime(year, month, date, hour, minute)
    return dt.strftime('%Y-%m-%d T%H:%M Z')

# Find correct format csv file of the data
# Parameter
# ==========================
# path: {string} the path of the directory that the csv file is in it
# output: {string} the name of the csv file
def find_csv_file(path):
    files = os.listdir(path)
    result = []
    for file in files:
        for item in re.findall('^weather.+.csv$', str(file)):
            result.append(item)
    if len(result) > 1:
        return result[-1]
    else:
        return result[0]
    return result



cols = ['Datetime', 'RAF', 'RA1', 'RI1', 'ERA', 'CRA', 'AT1', 'RH1', 'PRS', 'SLP', 'WD1', 'WDM', 'WS1', 
        'WSM', 'WND', 'WNS', 'SOL', 'WET', 'WBG', 'WEA']


# magnification of weather values
magni = [10, 10, 10, 10, 10, 10, 10, 10, 10, 1, 1, 10, 10, 1, 10, 1, 1, 10, 1]

root_path = '../../../data/poteka-raw-data/'

years = ['2019', '2020']
monthes = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
for observation_point in os.listdir(root_path):
    for year in years:
        for month in monthes:
            path = root_path + observation_point + f'/{year}/{month}/'
            # Check if the observation point has the data of year/month
            if os.path.exists(path):
                for date in os.listdir(path):
                    csv_path = path + f'{date}/Weather/'
                    #  Check if the observation point has the data of year/month/date and csv file of the data
                    if os.path.exists(csv_path) and len(os.listdir(csv_path)) > 0:
                        print('-'*50)
                        print(csv_path)
                        filename = find_csv_file(csv_path)
                        df_before = pd.read_csv(csv_path + filename, header=None)
                        print(df_before.info())

                        # Extract number of the data from the cell's value
                        for col in cols:
                            for col_def in df_before.columns:
                                if col in str(df_before[col_def].values[0]):
                                    df_before[col] = df_before[col_def].apply(find_value)
                                    df_before[col] = df_before[col].apply(float)
                        df_before.dropna(inplace=True)

                        # Extract datetime and format correctlly
                        for col in df_before.columns:
                            val = df_before[col].values[0]
                            check = find_value(str(val))
                            if len(check) == 14:
                                y = int(str(val)[:4])
                                m = int(str(val)[4:6])
                                d = int(str(val)[6:8])
                                df_before['Datetime'] = df_before[col].apply(str)
                                df_before['Datetime'] = df_before['Datetime'].apply(format_datetime)

                        df_before = df_before[cols]
                        df_before = df_before.set_index('Datetime')

                        # Rescale data
                        for i in range(len(cols[1:])):
                            col = cols[i+1]
                            mag = magni[i]
                            df_before[col] = df_before[col] / mag

                        # Reset index if there is a lack of the data
                        time_list = create_time_list(y, m, d)
                        df_before = df_before.drop_duplicates()
                        print(len(time_list), len(df_before.index))
                        print(df_before.info())
                        df = df_before.reindex(time_list)

                        save_path = '../../../data/cleaned-data/'
                        oymd = [observation_point, year, month, date]
                        for item in oymd:
                            save_path += f'{str(item)}/'
                            if not os.path.exists(save_path):
                                os.mkdir(save_path)


                        df.to_csv(save_path + 'data.csv')



--------------------------------------------------
../../../data/poteka-raw-data/Vlz-Bagbaguin_00174731/2019/03/2019-03-24/Weather/
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1440 entries, 0 to 1439
Data columns (total 21 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       1440 non-null   int64 
 1   1       1440 non-null   int64 
 2   2       1440 non-null   object
 3   3       1440 non-null   object
 4   4       1440 non-null   object
 5   5       1440 non-null   object
 6   6       1440 non-null   object
 7   7       1440 non-null   object
 8   8       1440 non-null   object
 9   9       1440 non-null   object
 10  10      1440 non-null   object
 11  11      1440 non-null   object
 12  12      1440 non-null   object
 13  13      1440 non-null   object
 14  14      1440 non-null   object
 15  15      1440 non-null   object
 16  16      1440 non-null   object
 17  17      1440 non-null   object
 18  18      1440 non-null   object
 19  

ParserError: Error tokenizing data. C error: Expected 21 fields in line 573, saw 31


In [95]:
path = '../../../data/poteka-raw-data/NagNayon_00181273/2020/04/2020-04-11/Weather/'
csv_file = find_csv_file(path)
df_before = pd.read_csv(path + csv_file, error_bad_lines=False, header=None)

b'Skipping line 573: expected 21 fields, saw 31\n'


In [96]:
df_before.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990 entries, 0 to 989
Data columns (total 21 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       990 non-null    int64  
 1   1       989 non-null    float64
 2   2       989 non-null    object 
 3   3       989 non-null    object 
 4   4       989 non-null    object 
 5   5       989 non-null    object 
 6   6       989 non-null    object 
 7   7       989 non-null    object 
 8   8       989 non-null    object 
 9   9       989 non-null    object 
 10  10      989 non-null    object 
 11  11      989 non-null    object 
 12  12      989 non-null    object 
 13  13      989 non-null    object 
 14  14      989 non-null    object 
 15  15      989 non-null    object 
 16  16      989 non-null    object 
 17  17      989 non-null    object 
 18  18      989 non-null    object 
 19  19      989 non-null    object 
 20  20      989 non-null    object 
dtypes: float64(1), int64(1), object(19)
mem

In [117]:
for col in cols:
    for col_def in df_before.columns:
        if col in str(df_before[col_def].values[0]):
            df_before[col] = df_before[col_def].apply(find_value)
            df_before[col] = df_before[col].apply(float)
df_before.dropna(inplace=True)

# Extract datetime and format correctlly
for col in df_before.columns:
    val = df_before[col].values[1]
    check = find_value(str(val).replace('.0', ''))
    if len(check) == 14:
        y = int(str(val)[:4])
        m = int(str(val)[4:6])
        d = int(str(val)[6:8])
        df_before['Datetime'] = df_before[col].apply(str)
        df_before['Datetime'] = df_before['Datetime'].apply(format_datetime)

df_before = df_before[cols]
df_before = df_before.set_index('Datetime')

In [118]:

df_before.head()

,RAF,RA1,RI1,ERA,CRA,AT1,RH1,PRS,SLP,WD1,WDM,WS1,WSM,WND,WNS,SOL,WET,WBG,WEA
Datetime,,,,,,,,,,,,,,,,,,,
2020-04-11 T00:00 Z,0.0,0.0,0.0,0.0,0.0,291.0,600.0,10104.0,10167.0,349.0,354.0,2.0,5.0,83.0,3.0,195.0,0.0,280.0,2.0
2020-04-11 T00:01 Z,0.0,0.0,0.0,0.0,0.0,293.0,598.0,10105.0,10168.0,327.0,2.0,2.0,7.0,62.0,3.0,247.0,0.0,281.0,0.0
2020-04-11 T00:02 Z,0.0,0.0,0.0,0.0,0.0,294.0,596.0,10105.0,10168.0,355.0,46.0,2.0,6.0,33.0,3.0,317.0,0.0,282.0,0.0
2020-04-11 T00:03 Z,0.0,0.0,0.0,0.0,0.0,295.0,594.0,10105.0,10168.0,229.0,234.0,2.0,5.0,359.0,2.0,290.0,0.0,283.0,0.0
2020-04-11 T00:04 Z,0.0,0.0,0.0,0.0,0.0,297.0,598.0,10105.0,10168.0,347.0,38.0,4.0,9.0,348.0,3.0,284.0,0.0,285.0,0.0
